In [1]:
#!pip install chart_studio
import common
import pandas as pd
import haversine as hs


In [2]:
df = common.df_from_list()
df.to_hdf('df.h5', 'df')
df

,datetime,duration,sla,slo,ela,elo
0,2023-01-03 12:11:00,9.650000,49.197684,16.640210,49.189396,16.617106
1,2023-01-03 17:57:55,11.550000,49.189396,16.617106,49.201914,16.641787
2,2023-01-04 11:48:46,10.550000,49.201914,16.641787,49.191415,16.619807
3,2023-01-04 17:53:59,12.533333,49.189396,16.617106,49.201914,16.641787
4,2023-01-09 11:49:58,11.033333,49.201914,16.641787,49.191415,16.619807
...,...,...,...,...,...,...
406,2024-05-07 12:36:15,2.550000,49.189396,16.617106,49.191415,16.619807
407,2024-05-07 13:27:16,7.266667,49.191415,16.619807,49.196597,16.608866
408,2024-05-07 18:16:10,13.450000,49.196597,16.608866,49.195861,16.650252
409,2024-05-08 10:14:22,11.533333,49.195861,16.650252,49.200186,16.631748


In [26]:
df.duration.idxmax()
df.take([138])

,datetime,duration,sla,slo,ela,elo,distance,map_distance
144,2023-05-04 12:55:31,326.65,49.2,16.64,49.2,16.61,2.46,3.35


In [27]:
# we use haversine package to calculate the distance between two points
df['distance'] = df.apply(lambda x: hs.haversine((x['sla'], x['slo']), (x['ela'], x['elo']), unit=hs.Unit.KILOMETERS), axis=1)

# I measured that travel distance is 1.36 times the aerial distance
df['map_distance'] = df.distance * 1.36


# Cleanup

# I never drive more than 50 km on nextbike
df = df[df.distance < 50]
df = df[df.distance > 0]

# I never spent more than 2 hours on nextbike
df = df[df.duration < 2 * 60]



In [34]:
pd.set_option('display.precision', 2)

print("Total distance travelled:  %.2f" % df.map_distance.sum())
print("Total time spent: %.2f" % (df.duration.sum() / 60), "hours")
print("Total trips:", df.shape[0])

Total distsncevs travelled:  1112.17
Total time spent: 75.46 hours
Total trips: 402


In [29]:
day_agg = common.time_agg(df, 'D', "%Y-%m-%d")
day_agg.index.rename('date', inplace=True)

print("Average distance per day: %.2f" % day_agg.distance.mean())
print("Average number of trips per day: %.2f" % day_agg.trips.mean())
print("Average time spent on bike per day: %.2f" % day_agg.duration.mean(), "min")
day_agg


Average distance per day: 4.92
Average number of trips per day: 1.78
Average time spent on bike per day: 20.03 min


,distance,duration,trips
date,,,
2023-01-03,5.69,21.20,2
2023-01-04,5.78,23.08,2
2023-01-09,5.78,24.05,2
2023-01-10,4.36,18.05,2
2023-01-11,6.40,26.72,3
...,...,...,...
2024-04-29,7.48,29.33,2
2024-04-30,3.17,11.28,1
2024-05-07,9.64,40.28,4


In [39]:
import requests

def points(start_lat,start_lng,end_lat,end_lng):
    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    querystring = {"waypoints":f"{start_lat},{start_lng}|{end_lat},{end_lng}","mode":"bicycle"}
    headers = {
        "X-RapidAPI-Key": "<your_key>",
        "X-RapidAPI-Host": "route-and-directions.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)

    return response

In [47]:
i = df.loc[10]
r= points(i.sla, i.slo, i.ela, i.elo)
r


<Response [403]>

In [58]:
pts = [((i.sla, i.slo), (i.ela, i.elo)) for i in df.itertuples()]

In [59]:
pts

[((49.197683515314, 16.640210151672), (49.189396089917, 16.617105603218)),
 ((49.189396089917, 16.617105603218), (49.201914334583, 16.641787290573)),
 ((49.201914334583, 16.641787290573), (49.191415398225, 16.619807185655)),
 ((49.189396089917, 16.617105603218), (49.201914334583, 16.641787290573)),
 ((49.201914334583, 16.641787290573), (49.191415398225, 16.619807185655)),
 ((49.189396089917, 16.617105603218), (49.201914334583, 16.641787290573)),
 ((49.189396089917, 16.617105603218), (49.200185532089, 16.631748274443)),
 ((49.200185532089, 16.631748274443), (49.189396089917, 16.617105603218)),
 ((49.201914334583, 16.641787290573), (49.199692060391, 16.627576947212)),
 ((49.199692060391, 16.627576947212), (49.189396089917, 16.617105603218)),
 ((49.189396089917, 16.617105603218), (49.201914334583, 16.641787290573)),
 ((49.189396089917, 16.617105603218), (49.201914334583, 16.641787290573)),
 ((49.201914334583, 16.641787290573), (49.189396089917, 16.617105603218)),
 ((49.201914334583, 16.64

In [67]:
import folium
import pandas as pd
m = folium.Map(tiles = 'Cartodb Positron')

def create_map(points,count=1):
    # use the response
    #coordinates = response['features'][0]['geometry']['coordinates']
    #points = [(i[1], i[0]) for i in coordinates[0]]
    coordinates = []
    for p in points:
       coordinates.append(p[0])
       coordinates.append(p[1])
    folium.PolyLine(points,color = 'red',  weight=.2, opacity=count).add_to(m)
    # create optimal zoom
    df = pd.DataFrame(coordinates).rename(columns={0:'Lat', 1:'Lon'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    return m

In [68]:
create_map(pts)